In [6]:
import google.generativeai as genai
import pandas as pd
import io
from dotenv import load_dotenv
import os

load_dotenv()

API_KEY  = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=API_KEY)

file_path = r"DataPDF\level1\abi.pdf"
output_csv_path = r"extracted_data.csv"

prompt = """
You are an expert data entry AI. Your task is to analyze the provided PDF, which is a quotation, and extract the line items from the main table into a clean, valid CSV format.

**Instructions:**

1.  **Identify the Main Table**: Locate the table with columns "ลำดับ", "รายการ", "จำนวน", "หน่วย", "ราคา/หน่วย", and "ราคารวม".
2.  **Extract Data for Each Row**:
    *   **รายชื่อบริษัทและการติดต่อ**: Use the value "บริษัท ลูก้า แอสเซท จำกัด" for every row.
    *   **ลำดับ**: Extract the item number (e.g., "1", "1.1", "2.1").
    *   **รายการ**: Extract the item description.
    *   **จำนวน**: Extract the quantity. If blank, provide an empty value.
    *   **หน่วย**: Extract the unit. If blank, provide an empty value.
    *   **ราคาต่อหน่วย**: Extract the unit price. If blank, provide an empty value.
    *   **ราคารวม**: Extract the total price. If blank, provide an empty value.
3.  **Handle Hierarchical Rows**: Rows that act as titles (like "Dynamic Load Test" and "Seismic Test") must be included, but their numerical columns (จำนวน, ราคาต่อหน่วย, ราคารวม) must be left empty.
4.  **Ignore Other Information**: Do not include the final summary rows (like the final "ราคารวม", "ภาษีมูลค่าเพิ่ม", "ยอดรวมทั้งสิ้น"), document headers, footers, or notes ("หมายเหตุ") in the CSV output.
5.  **Output Format**:
    *   The output MUST be **only** the CSV data.
    *   Do not add any explanations, comments, or markdown formatting like ```csv.
    *   The first line must be the header: "รายชื่อบริษัทและการติดต่อ","ลำดับ","รายการ","จำนวน","หน่วย","ราคาต่อหน่วย","ราคารวม"
    *   Ensure every single row has exactly 7 fields, matching the header.
"""

model = genai.GenerativeModel(model_name="gemini-2.5-flash")
uploaded_file = genai.upload_file(path=file_path, display_name="PDF for Extraction")
response = model.generate_content([prompt, uploaded_file])

csv_data_string = response.text.strip()
if csv_data_string.startswith("```csv"):
    csv_data_string = csv_data_string[len("```csv"):].strip()
if csv_data_string.endswith("```"):
    csv_data_string = csv_data_string[:-len("```")].strip()

string_data_io = io.StringIO(csv_data_string)
df = pd.read_csv(string_data_io)
df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')

print(f"Successfully extracted data and saved to {output_csv_path}")
print("\n--- Extracted Data Preview ---")
print(df)
print("----------------------------")

Successfully extracted data and saved to extracted_data.csv

--- Extracted Data Preview ---
   รายชื่อบริษัทและการติดต่อ  ลำดับ  \
0  บริษัท ลูก้า แอสเซท จำกัด    1.0   
1  บริษัท ลูก้า แอสเซท จำกัด    1.1   
2  บริษัท ลูก้า แอสเซท จำกัด    1.2   
3  บริษัท ลูก้า แอสเซท จำกัด    NaN   
4  บริษัท ลูก้า แอสเซท จำกัด    2.0   
5  บริษัท ลูก้า แอสเซท จำกัด    2.1   
6  บริษัท ลูก้า แอสเซท จำกัด    NaN   

                                              รายการ  จำนวน หน่วย  \
0                                  Dynamic Load Test    NaN   NaN   
1  ค่าบริการทดสอบกำลังรับน้ำหนักของเสาเข็มโดยวิธี...    1.0   ต้น   
2                       ค่าบริการรถเครน+ลูกตุ้มเหล็ก    1.0   งาน   
3            รายงานผลการทดสอบ+รับรองผลโดยสามัญวิศวกร    1.0   งาน   
4                                       Seismic Test    NaN   NaN   
5  ค่าบริการทดสอบความสมบูรณ์ของเสาเข็มด้วยวิธี Se...  113.0   ต้น   
6            รายงานผลการทดสอบ+รับรองผลโดยสามัญวิศวกร    1.0   งาน   

  ราคาต่อหน่วย  ราคารวม  
0          NaN  

In [7]:
import google.generativeai as genai
import pandas as pd
import io
import os
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=API_KEY)

folder_path = r"DataPDF\level3"
output_excel_path = r"level-.xlsx"

prompt = """
You are a highly precise data extraction AI. Your mission is to convert the provided PDF quotation into a specific CSV format by treating the document as two distinct sections and then merging them.

**Execution Plan:**

1.  **Section 1: Header Information Extraction**
    *   First, identify the recipient's details at the top: 'เรียน' (Company), 'ที่อยู่' (Address), 'ATTN' (Attention), and 'โครงการ' (Project).
    *   You will create a separate CSV row for each of these pieces of information.

2.  **Section 2: Line Items Table Extraction**
    *   Next, identify the main table containing services with columns: "ลำดับ", "รายการ", "จำนวน", "หน่วย", "ราคา/หน่วย", "ราคารวม".
    *   Extract every row from this table, including rows that act as sub-headers (e.g., "Dynamic Load Test").
    *   **CRUCIAL: The "จำนวน" column must contain only numeric values (integers or decimals). If the value is not a number, leave it as an empty quoted field (""). Do not include any non-numeric text in this column.**

3.  **Section 3: Summary Extraction**
    *   At the end of the document, extract summary rows such as "ราคารวม", "ภาษีมูลค่าเพิ่ม 7%", and "ยอดรวมทั้งสิ้น" with their corresponding values.
    *   For these summary rows, place the summary label in the 'รายการ' column and the value in the 'ราคารวม' column. All other columns must be empty quoted fields ("").

4.  **Data Validation (CRUCIAL):**
    *   Before outputting the CSV, carefully check the context of the PDF to ensure that all extracted data is correct and matches the original document.
    *   If you detect any inconsistencies or errors in the extracted data, correct them before generating the CSV.

5.  **CSV Construction Rules (Crucial):**
    *   The final output MUST be a single CSV text block.
    *   The header must be exactly: "รายชื่อบริษัทและการติดต่อ","ลำดับ","รายการ","จำนวน","หน่วย","ราคาต่อหน่วย","ราคารวม"
    *   **CRUCIAL QUOTING RULE**: Every single field in every row **MUST** be enclosed in double quotes (""). This is mandatory to handle commas within text fields. Example: "Value with, a comma","Value 2","","Value 4"
    *   **For Section 1 Rows (Header Info):**
        *   Place the extracted text It is the name of the company that created the quotation that is on the first letterhead. (e.g., "บริษัท เอบีไอ เทสติ้ง เอ็นจิเนียริ่ง จำกัด ABI  TESTING  ENGINEERING  CO.,LTD สำนักงานใหญ่เลขที่ 9/317 หมู่ที่ 3 ตำบลบางขนุน อำเภอบางกรวย จังหวัดนนทบุรี 11130 มือถือ 096 991 5545
Email : abitestingeng@gmail.com เลขประจำตัวผู้เสียภาษี  0 1255 66040 07 1") in the first column (`รายชื่อบริษัทและการติดต่อ`).
        *   All other six columns for these rows MUST be empty fields (represented as "").
    *   **For Section 2 Rows (Line Items):**
        *   The first column (`รายชื่อบริษัทและการติดต่อ`) MUST be an empty field ("").
        *   Populate columns 2 through 7 with the data extracted from the table. If a cell is empty, it must be represented as an empty quoted field ("").
    *   **For Section 3 Rows (Summary):**
        *   The first, second, fourth, fifth, and sixth columns MUST be empty fields ("").
        *   Place the summary label (e.g., "ราคารวม", "ภาษีมูลค่าเพิ่ม 7%", "ยอดรวมทั้งสิ้น") in the third column (`รายการ`).
        *   Place the summary value in the seventh column (`ราคารวม`).

6.  **Final Output Constraints**:
    *   Your entire response must contain **ONLY the raw CSV data** and nothing else.
    *   Do not include any explanations, summaries, or markdown formatting like ```csv.
"""

all_dfs = []
for filename in os.listdir(folder_path):
    if filename.lower().endswith(".pdf"):
        file_path = os.path.join(folder_path, filename)
        uploaded_file = genai.upload_file(path=file_path, display_name=f"PDF for Extraction: {filename}")
        response = genai.GenerativeModel(model_name="gemini-2.5-flash").generate_content([prompt, uploaded_file])
        csv_data_string = response.text.strip()
        if csv_data_string.startswith("```csv"):
            csv_data_string = csv_data_string[len("```csv"):].strip()
        if csv_data_string.endswith("```"):
            csv_data_string = csv_data_string[:-len("```")].strip()
        string_data_io = io.StringIO(csv_data_string)
        df = pd.read_csv(string_data_io, quotechar='"')
        all_dfs.append(df)

if all_dfs:
    merged_df = pd.concat(all_dfs, ignore_index=True)
    merged_df.to_excel(output_excel_path, index=False)
    print(f"Successfully extracted and merged data from all PDFs to {output_excel_path}")
    print("\n--- Extracted Data Preview ---")
    print(merged_df.to_string())
    print("----------------------------")
else:
    print("No PDF files found or extracted.")

KeyboardInterrupt: 

In [ ]:
import google.generativeai as genai
import pandas as pd
import io
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=API_KEY)

item_data = """
Product 1: Printer, qty 2
Product 2: Ink cartridge, qty 5
Supplier A: Printer 12000/unit, Ink 500/unit
Supplier B: Printer 11500/unit, Ink 550/unit
Supplier A Terms: Delivery 7 days, full payment after delivery
Supplier B Terms: Delivery 5 days, 30-day credit
"""

prompt = f"""
You are a data formatting assistant using LLM to transform unstructured quotation data into a standardized quotation table as per the attached layout.

Your task:
- Convert itemized data with multiple suppliers into a formatted table.
- The "ลำดับ" (Item No) column must be a running number starting from 1, 2, 3, ... according to the order in the file, not the numbering in the original document.
- Include columns: ลำดับ, รายการ, จำนวน, หน่วย, ลำดับที่1, ลำดับที่2, ลำดับที่3, ลำดับที่4, ลำดับที่5, ลำดับที่6.
- Each "ลำดับที่1" to "ลำดับที่6" column represents the price from each supplier (Supplier 1–6).
- Calculate TOTAL, VAT (7%), and TOTAL INC.VAT for each supplier column.
- At the end of the table, include the supplier conditions (delivery time, payment terms, etc.) in the specified section.

Input Example:
{item_data}

Output Format:
- The table must match the layout in the attached image exactly.
- The header row must be: ลำดับ	รายการ	จำนวน	หน่วย	ลำดับที่1	ลำดับที่2	ลำดับที่3	ลำดับที่4	ลำดับที่5	ลำดับที่6
- Each item row must have a running number in the "ลำดับ" column, starting from 1.
- Leave empty cells as blank (do not use "NA" or "-").
- After the last item, add three summary rows: TOTAL, vat, TOTAL INC.VAT (with calculated values for each supplier column).
- At the bottom, add the section "เงื่อนไขการของ SUPPLIER" and list the following rows:
  1	กำหนดยืนราคา
  2	ระยะเวลาส่งมอบสินค้าหลังจากได้รับ PO
  3	การชำระเงิน
  4	อื่น ๆ

The final output must be a plain text table (tab-separated) matching the procurement document layout, with all columns and summary/condition rows as shown in
"""

model = genai.GenerativeModel(model_name="gemini-2.5-flash")
response = model.generate_content(prompt)
output_text = response.text.strip()

with open("extracted_data_final.xlsx", "w", encoding="utf-8") as f:
    f.write(output_text)

print(output_text)

ลำดับ	รายการ	จำนวน	หน่วย	ลำดับที่1	ลำดับที่2	ลำดับที่3	ลำดับที่4	ลำดับที่5	ลำดับที่6
1	Printer	2	unit	12000	11500				
2	Ink cartridge	5	unit	500	550				
TOTAL					26500	25750				
vat					1855.00	1802.50				
TOTAL INC.VAT					28355.00	27552.50				

เงื่อนไขการของ SUPPLIER
1	กำหนดยืนราคา						
2	ระยะเวลาส่งมอบสินค้าหลังจากได้รับ PO	7 days	5 days			
3	การชำระเงิน	full payment after delivery	30-day credit			
4	อื่น ๆ


In [15]:
import os
import csv
import io
import gspread
from google.oauth2.service_account import Credentials
from openpyxl.utils import get_column_letter
from dotenv import load_dotenv

SCOPES = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive.file"
]
CREDS_FILE = 'glassy-keyword-466817-t8-8bdc0c7acadc.json'
SHEET_NAME = '17tMHStXQYXaIQHQIA4jdUyHaYt_tuoNCEEuJCstWEuw'
CSV_FILE_PATH = 'level-1.csv'
COMPANY_NAME_ROW = 1
CONTACT_INFO_ROW = 2
ITEM_MASTER_LIST_COL = 2
COLUMNS_PER_SUPPLIER = 4

load_dotenv()

def authenticate_and_open_sheet(sheet_name):
    creds = Credentials.from_service_account_file(CREDS_FILE, scopes=SCOPES)
    client = gspread.authorize(creds)
    spreadsheet = client.open_by_key(sheet_name)
    return spreadsheet.get_worksheet(0)

def parse_quotation_data(raw_data_string):
    quotations = []
    current = None
    f = io.StringIO(raw_data_string)
    reader = csv.reader(f)
    header = next(reader)
    column_map = {name: i for i, name in enumerate(header)}
    for row in reader:
        if len(row) < len(header):
            continue
        supplier_info = row[column_map["รายชื่อบริษัทและการติดต่อ"]].strip()
        item_no = row[column_map["ลำดับ"]].strip()
        item_desc = row[column_map["รายการ"]].strip()
        qty = row[column_map["จำนวน"]].strip()
        unit = row[column_map["หน่วย"]].strip()
        unit_price = row[column_map["ราคาต่อหน่วย"]].strip()
        total_price = row[column_map["ราคารวม"]].strip()
        if supplier_info:
            if current:
                quotations.append(current)
            if 'โทรศัพท์' in supplier_info:
                parts = supplier_info.split('โทรศัพท์')
                company_name = parts[0].strip()
                contact_info = parts[1].strip()
            elif 'โทร.' in supplier_info:
                parts = supplier_info.split('โทร.')
                company_name = parts[0].strip()
                contact_info = parts[1].strip()
            else:
                company_name = supplier_info
                contact_info = ""
            current = {
                "supplier_name": company_name,
                "contact_info": contact_info,
                "items": [],
                "summary": {}
            }
            continue
        if not current:
            continue
        if any(keyword in item_desc for keyword in ["ราคารวม", "ภาษีมูลค่าเพิ่ม", "ยอดรวมทั้งสิ้น"]):
            current["summary"][item_desc] = total_price
        elif item_desc:
            current["items"].append({
                "ลำดับ": item_no,
                "รายการ": item_desc,
                "ปริมาณ": qty,
                "หน่วย": unit,
                "ราคาต่อหน่วย": unit_price,
                "ราคารวม": total_price
            })
    if current:
        quotations.append(current)
    return quotations

def find_next_supplier_column(worksheet):
    existing_data = worksheet.row_values(COMPANY_NAME_ROW)
    col = ITEM_MASTER_LIST_COL + 1
    while col < len(existing_data) and existing_data[col]:
        col += COLUMNS_PER_SUPPLIER
    return col + 1

def build_and_update_data(worksheet, quote_data):
    start_col = find_next_supplier_column(worksheet)
    payloads = []
    payloads.append({
        'range': f"{get_column_letter(start_col)}{COMPANY_NAME_ROW}",
        'values': [[quote_data["supplier_name"]]]
    })
    payloads.append({
        'range': f"{get_column_letter(start_col)}{CONTACT_INFO_ROW}",
        'values': [[quote_data["contact_info"]]]
    })
    payloads.append({
        'range': f"{get_column_letter(start_col)}3:{get_column_letter(start_col + 3)}3",
        'values': [["ปริมาณ", "หน่วย", "ราคาต่อหน่วย", "รวมเป็นเงิน"]]
    })
    existing_items = worksheet.col_values(ITEM_MASTER_LIST_COL)
    for item in quote_data["items"]:
        item_name = item["รายการ"]
        if item_name in existing_items:
            row_index = existing_items.index(item_name) + 1
        else:
            existing_items.append(item_name)
            row_index = len(existing_items)
            payloads.append({
                'range': f"B{row_index}",
                'values': [[item_name]]
            })
        payloads.append({
            'range': f"{get_column_letter(start_col)}{row_index}:{get_column_letter(start_col + 3)}{row_index}",
            'values': [[item["ปริมาณ"], item["หน่วย"], item["ราคาต่อหน่วย"], item["ราคารวม"]]]
        })
    summary_start_row = len(existing_items) + 2
    for summary_key, summary_value in quote_data["summary"].items():
        payloads.append({
            'range': f"B{summary_start_row}",
            'values': [[summary_key]]
        })
        payloads.append({
            'range': f"{get_column_letter(start_col + 3)}{summary_start_row}",
            'values': [[summary_value]]
        })
        summary_start_row += 1
    if payloads:
        worksheet.batch_update(payloads, value_input_option='USER_ENTERED')

if __name__ == '__main__':
    with open(CSV_FILE_PATH, mode='r', encoding='utf-8') as infile:
        raw_data = infile.read()
    structured_data = parse_quotation_data(raw_data)
    worksheet = authenticate_and_open_sheet(SHEET_NAME)
    for quote in structured_data:
        build_and_update_data(worksheet, quote)
    print(f"Successfully processed {len(structured_data)} supplier quotations")

Successfully processed 5 supplier quotations


In [5]:
import os
import csv
import io
import gspread
from google.oauth2.service_account import Credentials
from openpyxl.utils import get_column_letter
from dotenv import load_dotenv

SCOPES = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive.file"
]
CREDS_FILE = 'glassy-keyword-466817-t8-8bdc0c7acadc.json'
SHEET_NAME = '17tMHStXQYXaIQHQIA4jdUyHaYt_tuoNCEEuJCstWEuw'
CSV_FILE_PATH = 'level-1.csv'

COMPANY_NAME_ROW = 1
CONTACT_INFO_ROW = 2
ITEM_MASTER_LIST_COL = 2
COLUMNS_PER_SUPPLIER = 4

load_dotenv()

def authenticate_and_open_sheet(sheet_name):
    creds = Credentials.from_service_account_file(CREDS_FILE, scopes=SCOPES)
    client = gspread.authorize(creds)
    spreadsheet = client.open_by_key(sheet_name)
    return spreadsheet.get_worksheet(0)

def parse_quotation_data(raw_data_string):
    quotations = []
    current = None
    f = io.StringIO(raw_data_string)
    reader = csv.reader(f)
    header = next(reader)
    column_map = {name: i for i, name in enumerate(header)}
    
    for row in reader:
        if len(row) < len(header):
            continue
            
        supplier_info = row[column_map["รายชื่อบริษัทและการติดต่อ"]].strip()
        item_no = row[column_map["ลำดับ"]].strip()
        item_desc = row[column_map["รายการ"]].strip()
        qty = row[column_map["จำนวน"]].strip()
        unit = row[column_map["หน่วย"]].strip()
        unit_price = row[column_map["ราคาต่อหน่วย"]].strip()
        total_price = row[column_map["ราคารวม"]].strip()
        
        if supplier_info:
            if current:
                quotations.append(current)
            
            if 'โทรศัพท์' in supplier_info:
                parts = supplier_info.split('โทรศัพท์')
                company_name = parts[0].strip()
                contact_info = parts[1].strip()
            elif 'โทร.' in supplier_info:
                parts = supplier_info.split('โทร.')
                company_name = parts[0].strip()
                contact_info = parts[1].strip()
            else:
                company_name = supplier_info
                contact_info = ""
            
            current = {
                "supplier_name": company_name,
                "contact_info": contact_info,
                "items": [],
                "summary": {}
            }
            continue
            
        if not current:
            continue
            
        if any(keyword in item_desc for keyword in ["ราคารวม", "ภาษีมูลค่าเพิ่ม", "ยอดรวมทั้งสิ้น"]):
            current["summary"][item_desc] = total_price
        elif item_desc:
            current["items"].append({
                "ลำดับ": item_no,
                "รายการ": item_desc,
                "ปริมาณ": qty,
                "หน่วย": unit,
                "ราคาต่อหน่วย": unit_price,
                "ราคารวม": total_price
            })
    
    if current:
        quotations.append(current)
    return quotations

def find_next_supplier_column(worksheet):
    existing_data = worksheet.row_values(COMPANY_NAME_ROW)
    col = ITEM_MASTER_LIST_COL + 1
    while col < len(existing_data) and existing_data[col]:
        col += COLUMNS_PER_SUPPLIER
    return col + 1

def build_and_update_data(worksheet, quote_data):
    start_col = find_next_supplier_column(worksheet)
    
    payloads = []
    
    payloads.append({
        'range': f"{get_column_letter(start_col)}{COMPANY_NAME_ROW}",
        'values': [[quote_data["supplier_name"]]]
    })
    
    payloads.append({
        'range': f"{get_column_letter(start_col)}{CONTACT_INFO_ROW}",
        'values': [[quote_data["contact_info"]]]
    })
    
    payloads.append({
        'range': f"{get_column_letter(start_col)}3:{get_column_letter(start_col + 3)}3",
        'values': [["ปริมาณ", "หน่วย", "ราคาต่อหน่วย", "รวมเป็นเงิน"]]
    })
    
    existing_items = worksheet.col_values(ITEM_MASTER_LIST_COL)
    item_start_row = 4
    
    for item in quote_data["items"]:
        item_name = item["รายการ"]
        
        if item_name in existing_items:
            row_index = existing_items.index(item_name) + 1
        else:
            existing_items.append(item_name)
            row_index = len(existing_items)
            payloads.append({
                'range': f"B{row_index}",
                'values': [[item_name]]
            })
        
        payloads.append({
            'range': f"{get_column_letter(start_col)}{row_index}:{get_column_letter(start_col + 3)}{row_index}",
            'values': [[item["ปริมาณ"], item["หน่วย"], item["ราคาต่อหน่วย"], item["ราคารวม"]]]
        })
    
    summary_start_row = len(existing_items) + 2
    for summary_key, summary_value in quote_data["summary"].items():
        payloads.append({
            'range': f"B{summary_start_row}",
            'values': [[summary_key]]
        })
        payloads.append({
            'range': f"{get_column_letter(start_col + 3)}{summary_start_row}",
            'values': [[summary_value]]
        })
        summary_start_row += 1
    
    if payloads:
        worksheet.batch_update(payloads, value_input_option='USER_ENTERED')

if __name__ == '__main__':
    with open(CSV_FILE_PATH, mode='r', encoding='utf-8') as infile:
        raw_data = infile.read()
    
    structured_data = parse_quotation_data(raw_data)
    worksheet = authenticate_and_open_sheet(SHEET_NAME)
    
    for quote in structured_data:
        build_and_update_data(worksheet, quote)